# COMP5329 - Assignment 2

In [ ]:
# import google
import collections
import json
import re

import nltk
import numpy as np
import pandas
import pandas as pd

# import torchtext
import PIL.Image
import timm
import torch
import torch.nn as nn
import torchvision

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# MOUNT_PATH = '/content/drive'
# DRIVE_PATH = f'{MOUNT_PATH}/My Drive'
# PROJECT_PATH = DRIVE_PATH + "/Assignment 2"
PROJECT_PATH = "./"
IMG_PATH = f"{PROJECT_PATH}/data"
TRAIN_CSV_PATH = f"{PROJECT_PATH}/train.csv"
TEST_CSV_PATH = f"{PROJECT_PATH}/test.csv"

# google.colab.drive.mount(MOUNT_PATH)
torch.cuda.empty_cache()

## Dataset


### Train dataset

In [ ]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TRAIN_CSV_PATH, names=range(4), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[3] = self.df_data[3].fillna("")
        self.df_data[2] += self.df_data[3]
        self.df_data = self.df_data.drop(3, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "label", 2: "caption"}, axis=1).dropna()

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

### Test dataset

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pandas.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data[2] = self.df_data[2].fillna("")
        self.df_data[1] += self.df_data[2]
        self.df_data = self.df_data.drop(2, axis=1)
        self.df_data = self.df_data.rename({0: "image", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        caption = self.df_data.iloc[idx, 1]

        sample = {"caption": caption, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample

## Preprocessing

### Dataset pre-transformations 

In [ ]:
def one_hot_encode_labels(dataset):
    if "one_hot_encoded_labels" not in dataset.tags:
        dataset.df_data["label"] = dataset.df_data["label"].apply(
            lambda l: torch.nn.functional.one_hot(
                torch.tensor([int(i) - 1 if int(i) < 12 else int(i) - 2 for i in l.split(" ")]), 18
            )
            .sum(axis=0)
            .float()
        )

        dataset.tags.add("one_hot_encoded_labels")

## Modules


### Caption embedding

In [ ]:
from transformers import AutoFeatureExtractor, AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
lang_model = AutoModel.from_pretrained("prajjwal1/bert-mini")

for name, param in lang_model.named_parameters():
    param.requires_grad_(False)

### Pretrained model surgery

In [ ]:
from effdet import create_model

img_model = create_model(
    "tf_efficientdet_d2", bench_task="predict", num_classes=18, pretrained=True
)
features = nn.Sequential(*[img_model.model.backbone, img_model.model.fpn])

for name, param in features.named_parameters():
    param.requires_grad_(False)
#     if 'fnode' in name :
#         if int(name.split(".")[4])>=5:
#             print(name)

In [ ]:
# for name, param in features.named_parameters():
#     print(name)
from torchinfo import summary

img = torch.randn(1, 3, 256, 256)
# summary(features, input_data=img, device="cpu")

In [ ]:
with torch.no_grad():
    out = features(img)

In [ ]:
for i in out:
    print(i.shape)

### Combined model

In [ ]:
NUM_ClASSES = 18
import math

import torch.nn as nn

IMG_DIM = out[0].shape[1]
WRD_DIM = list(lang_model.pooler.parameters())[-1].shape[0]


class SASGA(nn.Module):
    def __init__(self, word_dim=WRD_DIM):
        super().__init__()
        self.layer_norm_1 = norm = nn.LayerNorm(word_dim, eps=1e-05, elementwise_affine=True)
        self.layer_norm_2 = norm = nn.LayerNorm(word_dim, eps=1e-05, elementwise_affine=True)

        self.guided_attention_layer = nn.TransformerDecoderLayer(
            word_dim, nhead=word_dim // 64, dim_feedforward=word_dim * 4, activation="relu"
        )
        self.guided_attention = nn.TransformerDecoder(
            self.guided_attention_layer, num_layers=1, norm=self.layer_norm_1
        )
        self.self_attention_layer = nn.TransformerDecoderLayer(
            word_dim, nhead=word_dim // 64, dim_feedforward=word_dim * 4, activation="relu"
        )
        self.self_attention = nn.TransformerDecoder(
            self.self_attention_layer, num_layers=1, norm=self.layer_norm_2
        )

    def forward(self, x, y, y_mask=None):
        # guide x by y
        x = self.self_attention_layer(x, x)
        x = self.guided_attention(x, y, memory_key_padding_mask=y_mask)
        return x


class Combined_Model(torch.nn.Module):
    def __init__(self, visual_features, lang_model, word_dim=WRD_DIM, img_dim=IMG_DIM, n_saga=1):
        super().__init__()
        self.visual_features = visual_features
        self.lang_model = lang_model
        self.img_dim = img_dim
        self.word_dim = word_dim

        self.proj = nn.Linear(img_dim, 256)  # to word dims
        # SA-GA ##############################
        self.SASGA = nn.ModuleList([SASGA(word_dim) for _ in range(n_saga)])
        # SA-GA ##############################

        self.output = nn.Linear(word_dim, NUM_ClASSES)
        self.activation = nn.ReLU()
        self.proj_all = nn.Linear(word_dim * 2, word_dim)

    def forward(self, images, inputs):  # requires tokenized captions
        lang_outs = self.lang_model(**inputs)
        last_hidden_state = lang_outs["last_hidden_state"].permute(1, 0, 2)  # seq_len,
        pooler_out = lang_outs["pooler_output"]
        mask = inputs.attention_mask == 0

        features = self.visual_features(images)[-3:]
        batch_size = features[0].shape[0]
        features = torch.cat(
            [feature.view(batch_size, self.img_dim, -1) for feature in features], dim=-1
        )
        # batch, self.img_dim, -1

        features = self.proj(features.permute(2, 0, 1))
        for modules in self.SASGA:
            features = modules(features, last_hidden_state, mask)

        att_out = features.mean(0)  # batch, 256
        out = torch.cat([att_out, pooler_out], dim=-1)
        out = self.proj_all(out)
        out = self.activation(out)
        out = self.output(out)
        return out

In [ ]:
# for name, _ in model.module_list.named_parameters():
#     print(name, _.requires_grad)

## Transforms and training support

In [ ]:
class FieldTransform(object):
    def __init__(self, field, transform):
        self.field = field
        self.transform = transform

    def __call__(self, sample):
        sample[self.field] = self.transform(sample[self.field])
        return sample

## Experiments

In [ ]:
def train_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["label"] = torch.stack(X["label"])
    X["image"] = torch.stack(X["image"])

    return X


normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transforms = torchvision.transforms.Compose(
    [
        FieldTransform("image", torchvision.transforms.Resize((256, 256))),
        FieldTransform("image", torchvision.transforms.ToTensor()),
        FieldTransform("image", normalize),
    ]
)

In [ ]:
train_data = TrainDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(train_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
trainds, valds = random_split(train_data, [27000, 3000])

train_dl = DataLoader(
    trainds, batch_size=32, shuffle=True, collate_fn=train_collate_fn, num_workers=24
)

val_dl = DataLoader(valds, batch_size=4, shuffle=False, collate_fn=train_collate_fn, num_workers=24)


model = Combined_Model(features, lang_model, n_saga=1).to(device)


# from sam import SAM

# base_optimizer = torch.optim.Adam  # define an optimizer for the "sharpness-aware" update
# optimizer = SAM(model.parameters(), base_optimizer, lr=5e-5)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
criterion = torch.nn.BCEWithLogitsLoss()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.5,
    patience=2,
    threshold=0.0001,
    threshold_mode="rel",
    cooldown=0,
    min_lr=0,
    eps=1e-08,
    verbose=False,
)

In [ ]:
import os
from copy import deepcopy

from sklearn.metrics import f1_score
from tqdm import tqdm

# os.environ["TOKENIZERS_PARALLELISM"] = "true"


val_mean_f1 = []
val_sample_f1 = []
model_in_memory = {}

scaler = torch.cuda.amp.GradScaler()


def train_epoch(model, train_data_loader, use_fp16=False):
    train_loss = []
    model.train()
    for batch in tqdm(train_data_loader):
        optimizer.zero_grad()
        captions = batch["caption"]
        images = batch["image"].to(device)
        labels = batch["label"].to(device)
        inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

        with torch.cuda.amp.autocast(enabled=use_fp16):
            predictions = model(images, inputs)
            loss = criterion(predictions, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss.append(loss.item())
    return train_loss


def evaluate_model(model, eval_data_loader):
    val_loss = []
    val_outs = []
    val_labels = []
    model.eval()
    for batch in val_dl:
        captions = batch["caption"]
        images = batch["image"].to(DEVICE)
        labels = batch["label"].to(DEVICE)
        inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)
        predictions = model(images, inputs)
        loss = criterion(predictions, labels)

        val_loss.append(loss.item())
        val_outs.append(predictions.detach().cpu().numpy())
        val_labels.append(labels.detach().cpu().numpy())
    val_labels = np.vstack(val_labels)
    val_outs = np.vstack(val_outs)
    mean_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="micro")
    sample_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs > 0), average="samples")
    return val_loss, val_outs, val_labels, mean_f1, sample_f1


for epoch in range(50):
    train_loss = []
    train_outs = []
    train_lables = []
    model.train()

    if epoch == 2:  #
        finetune_param = False
        for name, param in model.visual_features.named_parameters():
            if "fnode" in name:
                #                 if int(name.split(".")[4])>=5: # finetune last 3 layers
                param.requires_grad_(True)

    if epoch == 4:
        finetune_param = False
        for name, param in model.lang_model.named_parameters():
            if "3" in name:
                finetune_param = True
            if finetune_param:
                param.requires_grad_(True)

    train_loss = train_epoch(model, train_dl, use_fp16=True)
    val_loss, val_outs, val_labels, mean_f1, sample_f1 = evaluate_model(model, val_dl)

    cur_lr = optimizer.param_groups[0]["lr"]
    print(
        f"Epoch:{epoch}, train/val loss:{round(np.mean(train_loss),7),round(np.mean(val_loss),7)},sample/mean f1:{round(sample_f1, 7), round(mean_f1, 7)}, lr:{cur_lr}"
    )
    scheduler.step(np.mean(val_loss))

    if len(val_sample_f1) == 0 or sample_f1 > max(val_sample_f1):
        print("saving best")
        try:
            del model_in_memory["best"]
        except:
            pass
        torch.cuda.empty_cache()
        model.to("cpu")
        model_in_memory["best"] = deepcopy(model.state_dict())
        model.to(device)
    if epoch % 10 == 0:
        print("saving best to disk")
        torch.save(model_in_memory["best"], "test_v10.pt")

    val_mean_f1.append(mean_f1)
    val_sample_f1.append(sample_f1)
    
    #"manual early stopping"
    if cur_lr < 3e-6:
        torch.save(model_in_memory["best"], "test_v10.pt")
        break

###################

In [ ]:
raise ValueError
# (finetune last 3 layer (15-17)) + (SAM, lr 0.001)
# b0 + bert tiny + no attention = 85
# b0 + bert tiny + SAGA * 1 = 85 ...
# b0 + bert tiny + bmm (attention but no weights) at last conv filters = fails..?! (lr too high )
# b0 (13-) + bert tiny + bmm (attention but no weights) = 85
# b0 + bert tiny (both no finetune) + SAGA * 1 (5e-5 lr)= 86.40
# b2 + bert mini (both no finetune) + SAGA * 1 (5e-5 lr)= 86.3
# b2 + bert mini (finetune from 21/3 attention) + SAGA * 1 (5e-5 lr)= 87.4..
# d2 (2; fnodes 5) + bert mini (4; 3 attention) + 1*SAGA (1 wrd_dim) + adam=5e-5 batch=16 fp16 = 89.12271
# d2 (2; fnodes all) + bert mini (4; 3 attention) + 1*SAGA (4 wrd_dim) + adam=5e-5 batch=16 fp16 = 89.22668

# correction

In [ ]:
# model_name = "test.pt"
# model = LUKE_CRF(tag_to_ix, lang_model=lang_model, hidden_dim=1024)
model = Combined_Model(features, lang_model, n_saga=1)
model.load_state_dict(torch.load("test_v10.pt"))
model.to(device)
# model = torch.load(model_name)

In [ ]:
# cheat
class CheatDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.tags = set()
        self.df_data = pd.read_csv("test_cheat.csv", names=range(3), skiprows=1)[[0, 2, 1]].dropna()
        self.df_data[0] = IMG_PATH + "/" + self.df_data[0]
        self.df_data = self.df_data.rename({0: "image", 2: "label", 1: "caption"}, axis=1)

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = PIL.Image.open(self.df_data.iloc[idx, 0])
        label = self.df_data.iloc[idx, 1]
        caption = self.df_data.iloc[idx, 2]

        sample = {"caption": caption, "label": label, "image": image}

        if self.transform:
            sample = self.transform(sample)

        return sample


cheat_data = CheatDataset(transform=transforms)
# vocabularise_caption(train_data, vocab)
one_hot_encode_labels(cheat_data)

from torch.utils.data import DataLoader, random_split

# train_data.__len__() == 30000
cheat_dl = DataLoader(
    cheat_data, batch_size=16, shuffle=False, collate_fn=train_collate_fn, num_workers=24
)

from sklearn.metrics import precision_recall_curve

# use the same validation set/training set

val_loss, val_outs, val_labels, mean_f1, sample_f1 = evaluate_model(model, cheat_dl)

best_thresholds = np.zeros(18)
for i in range(18):
    fpr, tpr, thresholds = precision, recall, thresholds = precision_recall_curve(
        val_labels[:, i], (val_outs)[:, i]
    )
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    best_thresholds[i] = thresholds[ix]

print(best_thresholds)
np.save("best_thresholds", best_thresholds)

new_f1 = f1_score(y_true=val_labels, y_pred=1 * (val_outs - best_thresholds) > 0, average="samples")
print(f"old_f1: {sample_f1}; new_f1:{new_f1}")

# generate test labels

In [ ]:
# stopper for run all
raise ValueError

In [25]:
def test_collate_fn(X):
    # convert [{key: val, ...}, ...]
    # to [key: [val, ...],  ...}
    X = {k: [v[k] for v in X] for k in X[0]}
    X["image"] = torch.stack(X["image"])

    return X


test_data = TestDataset(transform=transforms)
test_dl = torch.utils.data.DataLoader(
    test_data, batch_size=16, shuffle=False, collate_fn=test_collate_fn, num_workers=0,
)


# save predictions
model.eval()
test_preds = []
for i, batch in enumerate(test_dl):
    images = batch["image"].to(device)
    captions = batch["caption"]
    inputs = tokenizer(captions, return_tensors="pt", padding=True, truncation=False).to(device)

    predictions = model(images, inputs)
    test_preds.append(predictions.detach().cpu().numpy())

test_preds = np.vstack(test_preds)


def out_logits_to_preds(logits, best_thresholds, correction=False):
    labels = []
    #... correction doesnt work..??
    if correction:
        logits = logits - best_thresholds
    logits = (logits > 0)
    lables_available = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19])
    for i in range(logits.shape[0]):
        labels.append(list(lables_available[logits[i]]))
    return labels


# lables_available[(test_preds[:10] > 0)]
# {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}

test_labels = out_logits_to_preds(test_preds, best_thresholds)

df_test = pd.read_csv(TEST_CSV_PATH, names=range(3), skiprows=1)
test_labels_str = [" ".join([str(i) for i in labels]) for labels in test_labels]
df_test["Labels"] = test_labels_str
df_test.rename({0: "ImageID"}, axis=1, inplace=True)

df_test[["ImageID", "Labels"]].to_csv("test_predictions.csv", index=False)
